In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
import plotly.graph_objects as go

Le Covid-19 peut être modélisé à l'aide d'un modèle de type Susceptible-Exposed-Infectious-Removed (SEIR).
Le package EoN (Epidemics on Networks) permet justement de créer ce genre d'estimation.

On estime qu'en France une personne adulte a en moyenne un contact avec 40 autres personnes et que le virus est à un taux de reproduction (R_0) estimé entre 1.4 et 3.2 (https://cmmid.github.io/topics/covid19/current-patterns-transmission/global-time-varying-transmission.html). 

Sa période d'incubation est de 5 jours et la période de contagion est d'environ 11 jours.


Le modèle SEIR est un modèle permettant de voir comment une maladie se répend dans la population. 

L'acronyme tient pour : Suceptible Exposed Infected Recovered (Suspicision Exposé Infecté Rétabli)


<center>
$\dot{S}  = \beta SI$ (1)
    
    Variation de personnes suceptible de tomber malade, modéré par le nombre de personnes infectés et leur contact avec les infectés
</center>
<center>
$\dot{E} = \beta SI - \alpha E $  (2)
    
    Nombre de personnes ayant été exposé à la maladie. Croissance basé sur le taux de contact mais diminuer par la période d'incubation 
</center>
<center>
$\dot{I} = \alpha E - \gamma I$ (3)
    
    Variation dans le nombre de personnes infectées basé sur la population exposée and la période d'incubation. Décroit en fonction de la période d'infection, donc plus $\gamma $ est élévée plus les gens meurent vite ou se restaurent vite ce qui nous envoie vers R le nombre de personne ayant été soigné/restauré.
</center>
<center>
$\dot{R} \gamma I$ (4)
    
    
</center>
<center>
$N = S + E + I +R$ (5)
    
    Contrainte pour l'assurer que la population reste identique ! 
</center>
3 paramètres : $\alpha,\beta,\gamma  $

$\alpha$ est l'inverse de la période d'incubation (1/incubation)

$\beta$ est le taux moyen de contact de la population

$\gamma$ est l'inverse de la période moyenne d'infection (1/infection)

Autre valeur importante : le $R_0$ qui montre à quelle vitesse la maladie se répend et peut être calculé de la manière suivante : 


<center>$R_0 = \frac{\beta}{\gamma}$ (6)</center>
    
A nous par la suite de définir une période d'incubation. Ajd, ca semble être 5 jours donc $\alpha$ semble être aux alentours de 0.2. 

Le $R_0$ est semblerait il d'après plusieurs papiers autour de 3.5 . A voir en France.

Et à l'aide d'un $\gamma$ défini nous pouvons avec (6) retrouvé $\beta$


### Ajout d'un effet de distance sociale

Cette fois ci , ajout d'un effet de social distancing qui va avoir un impact sur notre $\beta$ variable mesurant le taux de contact entre les personnes.

Soit $\rho$ la distanciation sociale variant entre 0 et 1. Où 0 est tout le monde est confiné, 1 tout le monde circule comme il le souhaite. Pour introduire ce dernier facteur, on va simplement changer nos deux premières équations en : 

<center> $\dot{S} = -\rho \beta SI$ (1') </center>

<center> $\dot{E} = \rho \beta SI - \alpha E $ (2') </center>

In [83]:
def SEIR_mano(val_init,coeff,t):
    """
        Input : 
            val_init : Liste de 4 valeurs initials que va prendre le modèle 
            coeff : Liste de valeurs pour Alpha,Gamma, Beta et Rho
            t :
        Output:
            
    """
    S0,E0,I0,R0= val_init
    S,E,I,R = [S0],[E0],[I0],[R0]
    a,b,g,r=coeff
    dt = t[1]-t[0]
    for _ in t[1:]:
        nS = S[-1] - (r*b*S[-1]*I[-1])*dt
        nE = E[-1] + (r*b*S[-1]*I[-1]-a*E[-1])*dt
        nI = I[-1] + (a*E[-1]-g*I[-1])*dt
        nR = R[-1] + (g*I[-1])*dt
        S.append(nS)
        E.append(nE)
        I.append(nI)
        R.append(nR)
    return np.stack([S,E,I,R]).T

In [84]:
def simulator(R0,N=6000,exposed = 1,rho =1,t = range(0,100),incub_time=5,infec_time=11):
    alpha = 1/incub_time
    gamma = 1/infec_time
    beta = R0 * gamma
    rho = 1
    N = 6000
    coeff = alpha, beta,gamma,rho
    init_vals = 1 - exposed/N,1/N,0,0
    df = pd.DataFrame(SEIR_mano(init_vals,coeff,t),columns=['S','E','I','R'])
    lst=[]
    for index,row in df.iterrows():
        lst+=[(round(row.S*N),'S',index)]
        lst+=[(round(row.E*N),'E',index)]
        lst+=[(round(row.I*N),'I',index)]
        lst+=[(round(row.R*N),'R',index)]
    return pd.DataFrame(lst,columns=['Nb','Type','Idx'])

In [85]:
simulator(3.2)

,Nb,Type,Idx
0,5999.0,S,0
1,1.0,E,0
2,0.0,I,0
3,0.0,R,0
4,5999.0,S,1
...,...,...,...
395,2423.0,R,98
396,1498.0,S,99
397,612.0,E,99
398,1346.0,I,99


In [86]:
df.shape

(244, 7)

In [87]:
lst=[]
for index,row in df.iterrows():
    lst+=[(round(row.S*N),'S',index)]
    lst+=[(round(row.E*N),'E',index)]
    lst+=[(round(row.I*N),'I',index)]
    lst+=[(round(row.R*N),'R',index)]

AttributeError: 'Series' object has no attribute 'S'

In [88]:
test = pd.DataFrame(lst,columns=['Nb','Type','Idx'])

In [89]:
import plotly.graph_objects as go
fig = go.Figure(data=[
    go.Bar(name='Exposed', x=test.Idx, y=test[test.Type == 'E'].Nb),
    go.Bar(name='Infected', x=test.Idx, y=test[test.Type == 'I'].Nb),
    go.Bar(name='Recovered', x=test.Idx, y=test[test.Type == 'R'].Nb),
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [90]:
import plotly.express as px
df = px.data.tips()
fig = px.bar(test, x="Idx", y="Nb", color='Type')
fig.show()

KeyError: 'Type'

In [91]:
x=t
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x, y=df.S,
    mode='lines',
    line=dict(width=0.5, color='rgb(184, 247, 212)'),
    stackgroup='one',
    groupnorm='percent' # sets the normalization for the sum of the stackgroup
))
fig.add_trace(go.Scatter(
    x=x, y=df.E,
    mode='lines',
    line=dict(width=0.5, color='rgb(111, 231, 219)'),
    stackgroup='one'
))
fig.add_trace(go.Scatter(
    x=x, y=df.I,
    mode='lines',
    line=dict(width=0.5, color='rgb(127, 166, 238)'),
    stackgroup='one'
))
fig.add_trace(go.Scatter(
    x=x, y=df.R,
    mode='lines',
    line=dict(width=0.5, color='rgb(131, 90, 241)'),
    stackgroup='one'
))

fig.update_layout(
    showlegend=True,
    xaxis_type='category',
    yaxis=dict(
        type='linear',
        range=[1, 100],
        ticksuffix='%'))

fig.show()

NameError: name 't' is not defined

In [92]:

# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for step in np.arange(0, 10, 0.1):
    fig.add_trace(go.Bar(name='Exposed',
            visible=False,
            x=simulator(step).Idx,
            y=simulator(step)[simulator(step).Type == 'E'].Nb))

# Make 3.2
fig.data[32].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
        label = '',
        
    )
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=32,
    currentvalue={"prefix": "<b>Taux de reproduction (R0):</b> "},
    pad={"t": 50},
    ticklen = 0,
    tickwidth = 0,
    steps=steps,
    len=0.25,
    transition= {'easing':"cubic"}
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

In [9]:
def SEIR_mano_augmented(val_init,coeff,jour_inter,t):
    """
        Input : 
            val_init : Liste de 4 valeurs initials que va prendre le modèle 
            coeff : Liste de valeurs pour Alpha,Gamma, Beta et Rho
            t :
        Output:
            
    """
    S0,E0,I0= val_init
    jour_inter = 100
    S,E,I,Mild,Severe,Severe_hosp,Fatal,R_mild,R_Severe,R_Fatal = [S0],[E0],[I0],[0],[0],[0],[0],[0],[0],[0]
    a,b,g,r,death_rate,D_death,p_severe,recov_mild,duree_hosp,recover_severe=coeff
    p_mild = 1 - death_rate - p_severe
    for _ in t[1:]:
        if _ < jour_inter:
            nS = S[-1] - (b*S[-1]*I[-1])
            nE = E[-1] + (b*S[-1]*I[-1]-a*E[-1])
            nI = I[-1] + (a*E[-1]-g*I[-1])
            nMild = Mild[-1] + (p_mild * g * I[-1] - (1/recov_mild) * Mild[-1]) 
            nSevere = Severe[-1] + (p_severe * g * I[-1]  - (1/duree_hosp ) * Severe[-1])
            nSevere_hosp = Severe_hosp[-1]+ ((1/duree_hosp)*Severe[-1] - (1/recover_severe) * Severe_hosp[-1])
            nFatal = Fatal[-1] + (death_rate*g*I[-1] - (1/D_death) *Fatal[-1])
            nR_mild = R_mild[-1] + ((1/recov_mild) * Mild[-1])
            nR_Severe = R_Severe[-1] + ((1/recover_severe)*Severe_hosp[-1])
            nR_Fatal = R_Fatal[-1] + ((1/D_death) * Fatal[-1])
            S.append(nS)
            E.append(nE)
            I.append(nI)
            Mild.append(nMild)
            Severe.append(nSevere)
            Severe_hosp.append(nSevere_hosp)
            Fatal.append(nFatal)
            R_mild.append(nR_mild)
            R_Severe.append(nR_Severe)
            R_Fatal.append(nR_Fatal)
        else:
            nS = S[-1] - (r*b*S[-1]*I[-1])
            nE = E[-1] + (r*b*S[-1]*I[-1]-a*E[-1])
            nI = I[-1] + (a*E[-1]-g*I[-1])
            nMild = Mild[-1] + (p_mild * g * I[-1] - (1/recov_mild) * Mild[-1]) 
            nSevere = Severe[-1] + (p_severe * g * I[-1]  - (1/duree_hosp ) * Severe[-1])
            nSevere_hosp = Severe_hosp[-1]+ ((1/duree_hosp)*Severe[-1] - (1/recover_severe) * Severe_hosp[-1])
            nFatal = Fatal[-1] + (death_rate*g*I[-1] - (1/D_death) *Fatal[-1])
            nR_mild = R_mild[-1] + ((1/recov_mild) * Mild[-1])
            nR_Severe = R_Severe[-1] + ((1/recover_severe)*Severe_hosp[-1])
            nR_Fatal = R_Fatal[-1] + ((1/D_death) * Fatal[-1])
            S.append(nS)
            E.append(nE)
            I.append(nI)
            Mild.append(nMild)
            Severe.append(nSevere)
            Severe_hosp.append(nSevere_hosp)
            Fatal.append(nFatal)
            R_mild.append(nR_mild)
            R_Severe.append(nR_Severe)
            R_Fatal.append(nR_Fatal)
    return np.stack([S,E,I,Mild,Severe,Severe_hosp,Fatal,R_mild,R_Severe,R_Fatal]).T

In [15]:
def simulator(R0,incub_time,infec_time,exposed,death_rate,death_time,p_severe,duree_hosp,N,rho,t,jour_inter):
    alpha = 1/incub_time
    gamma = 1/infec_time
    beta = R0 * gamma
    rho = 1
    recov_mild =  (14 - infec_time)
    recover_severe = (31.5 - infec_time)
    Time_to_death = death_time
    D_death = Time_to_death - infec_time
    coeff = alpha, beta,gamma,rho,death_rate,D_death,p_severe,recov_mild,duree_hosp,recover_severe
    init_vals = 1-exposed/N,exposed/N,0
    df = pd.DataFrame(SEIR_mano_augmented(init_vals,coeff,jour_inter,t),columns=['S','E','I','Mild','Severe','Severe_hosp','Fatal','R_Mild','R_Severe','R_Fatal'])
    lst=[]
    for index,row in df.iterrows():
        lst+=[(round(row.S*N),'S',index)]
        lst+=[(round(row.E*N),'E',index)]
        lst+=[(round(row.I*N),'I',index)]
        lst+=[(round(row.R_Fatal*N),'Death',index)]
        lst+=[(round(N * (row.Severe_hosp + row.Fatal)),'Hospital',index)]
        lst+=[(round(N * (row.R_Mild + row.R_Severe)),'Recovered',index)]
    return pd.DataFrame(lst,columns=['Nb','Type','Idx'])

In [16]:
sim = simulator(R0 = 3.2,incub_time = 5.2, infec_time = 2.9, exposed = 1, death_rate = 0.02, death_time = 32, p_severe = 0.2,duree_hosp = 5,N = 50000,rho=1,t=range(1,365),jour_inter=100)

In [130]:

# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for step in np.arange(0, 10, 0.1):
    fig.add_trace(go.Bar(name='Exposed',
            visible=False,
            x=sim.Idx.unique(),
            y=sim[sim.Type == 'E'].Nb))

# Make 3.2
fig.data[32].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
        label = '',
        
    )
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=32,
    currentvalue={"prefix": "<b>Taux de reproduction (R0):</b> "},
    pad={"t": 50},
    ticklen = 0,
    tickwidth = 0,
    steps=steps,
    len=0.25,
    transition= {'easing':"cubic"}
)]

fig.update_layout(
    sliders=sliders
)

fig.show()